#### Random forest uses eeg to classify stride phase

In [1]:
import numpy as np
import xarray as xr
import FFT_functions as fft_f
from sklearn import preprocessing as prep 
from sklearn.model_selection import TimeSeriesSplit
import torch.nn as nn
import torch.optim as optim

In [2]:
# Loading data for testing
Windows_path='/mnt/c/Users/heather/Desktop/Neuromotor/Data4ML'
eeg_path=Windows_path+'/EEG_ICA_filtered_10_components'
velocity_path=Windows_path+'/Velocity'

In [3]:
eeg_data=xr.load_dataset(eeg_path)
eeg_=eeg_data.isel(Trial=0).copy()
eeg=eeg_.__xarray_dataarray_variable__
eeg=np.array(eeg.values)

velocity_data=xr.load_dataset(velocity_path)
vel_=velocity_data.isel(Trial=0).copy()
velocity=vel_.__xarray_dataarray_variable__
velocity=np.array(velocity.values)

In [4]:

print(np.isnan(velocity).any())


False


In [5]:
def velocity_rescaler(data):
    min_max_scaler = prep.MinMaxScaler() 
    for i in range(data.shape[1]):
        channel=data[:,i]
        data_2d = channel.reshape(-1, 1)
        normalizedData = min_max_scaler.fit_transform(data_2d) 
        normalized_data = normalizedData.flatten()
        data[:,i]=normalized_data
    return data

In [6]:
velocity=velocity_rescaler(velocity)
print(np.max(velocity))

1.0


In [7]:
print(velocity.shape)

(90000, 6)


In [8]:
data,labels=fft_f.make_windows_with_fft(eeg,velocity,10,3,0,lowpass_cutoff=30,debug=True)

n_samples:  90000
n_features:  60
n_windows:  8999
len(eeg_windows):  8999
len(joint_windows):  8999
eeg_windows[0].shape (10, 60)
joint_windows[0].shape (3, 6)


In [9]:
batch_size = 1  
dataloaders, train_ind, test_ind, train_data, test_data, train_lab, test_lab = fft_f.create_time_series_dataloaders(data, labels, batch_size)

# Accessing the train and test data directly
x_train_list, x_test_list = train_data, test_data
y_train_list, y_test_list = train_lab, test_lab

# Example to check shapes
for i in range(len(x_train_list)):
    print(f"Split {i}:")
    print(f"x_train shape: {np.array(x_train_list[i]).shape}, y_train shape: {np.array(y_train_list[i]).shape}")
    print(f"x_test shape: {np.array(x_test_list[i]).shape}, y_test shape: {np.array(y_test_list[i]).shape}")
    print("-------------------------")

Split 0:
x_train shape: (1504, 60, 10, 2), y_train shape: (1504, 6)
x_test shape: (1499, 60, 10, 2), y_test shape: (1499, 6)
-------------------------
Split 1:
x_train shape: (3003, 60, 10, 2), y_train shape: (3003, 6)
x_test shape: (1499, 60, 10, 2), y_test shape: (1499, 6)
-------------------------
Split 2:
x_train shape: (4502, 60, 10, 2), y_train shape: (4502, 6)
x_test shape: (1499, 60, 10, 2), y_test shape: (1499, 6)
-------------------------
Split 3:
x_train shape: (6001, 60, 10, 2), y_train shape: (6001, 6)
x_test shape: (1499, 60, 10, 2), y_test shape: (1499, 6)
-------------------------
Split 4:
x_train shape: (7500, 60, 10, 2), y_train shape: (7500, 6)
x_test shape: (1499, 60, 10, 2), y_test shape: (1499, 6)
-------------------------


In [10]:
# Hyperparameters
input_size = 2  
hidden_size = 128
num_layers = 2
output_size = 1  
num_epochs = 20
learning_rate = 0.001

# Initialize model, criterion, and optimizer
model = fft_f.LSTM_model_maker(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_loader = dataloaders[0]

# Train the model
trained_lstm=fft_f.train_model(model, train_loader, num_epochs, criterion, optimizer)

/home/knockonhwood/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], Loss: 0.0028
